# 使用 AWS EC2 实例
:label:`sec_aws`

在本节中，我们将向您展示如何在原始 Linux 机器上安装所有库。回想一下，在 :numref:`sec_sagemaker` 中我们讨论了如何使用 Amazon SageMaker，而自己构建实例在 AWS 上的成本更低。此过程包括三个步骤：

1. 从 AWS EC2 请求 GPU Linux 实例。
1. 安装 CUDA（或使用预装了 CUDA 的 Amazon Machine Image）。
1. 安装深度学习框架和其他用于运行本书代码的库。

这一过程也适用于其他实例（和其他云），尽管可能需要进行一些小的修改。在继续之前，您需要创建一个 AWS 账户，请参阅 :numref:`sec_sagemaker` 以获取更多详细信息。

## 创建和运行 EC2 实例

登录您的 AWS 账户后，点击“EC2” (:numref:`fig_aws`) 进入 EC2 面板。

![打开 EC2 控制台。](../img/aws.png)
:width:`400px`
:label:`fig_aws`

:numref:`fig_ec2` 显示了 EC2 面板。

![EC2 面板。](../img/ec2.png)
:width:`700px`
:label:`fig_ec2`

### 预设位置
选择附近的数据中心以减少延迟，例如“俄勒冈”（:numref:`fig_ec2` 右上角红色框中标记）。如果您位于中国，
可以选择附近的亚太地区，如首尔或东京。请注意
某些数据中心可能没有 GPU 实例。

### 增加限制

在选择实例之前，通过点击左侧栏中的“Limits”标签来检查是否存在数量
限制，如 :numref:`fig_ec2` 所示。
:numref:`fig_limits` 显示了这种
限制的一个例子。当前账户无法根据该区域打开“p2.xlarge”实例。如果
需要打开一个或多个实例，点击“Request limit increase”链接以
申请更高的实例配额。
通常，处理申请需要一个工作日。

![实例数量限制。](../img/limits.png)
:width:`700px`
:label:`fig_limits`


### 启动实例

接下来，点击 :numref:`fig_ec2` 中红色框标记的“Launch Instance”按钮来启动您的实例。

我们首先选择一个合适的 Amazon Machine Image (AMI)。选择一个 Ubuntu 实例 (:numref:`fig_ubuntu`)。


![选择 AMI。](../img/ubuntu-new.png)
:width:`700px`
:label:`fig_ubuntu`

EC2 提供了许多不同的实例配置供选择。对于初学者来说，这有时可能会感到不知所措。:numref:`tab_ec2` 列出了不同的适合机器。

:不同类型的 EC2 实例
:label:`tab_ec2`

| 名称 | GPU         | 备注                         |
|------|-------------|-------------------------------|
| g2   | Grid K520   | 陈旧                       |
| p2   | Kepler K80  | 旧但通常作为现货便宜   |
| g3   | Maxwell M60 | 性价比好                |
| p3   | Volta V100  | 高性能 FP16     |
| p4   | Ampere A100 | 大规模训练高性能 |
| g4   | Turing T4   | 推理优化 FP16/INT8 |


所有这些服务器都有多种规格，表示使用的 GPU 数量。例如，p2.xlarge 有 1 个 GPU，而 p2.16xlarge 有 16 个 GPU 和更多的内存。有关更多详细信息，请参阅 [AWS EC2 文档](https://aws.amazon.com/ec2/instance-types/) 或 [摘要页面](https://www.ec2instances.info)。为了说明目的，p2.xlarge 就足够了（:numref:`fig_p2x` 中红色框标记）。

![选择实例。](../img/p2x.png)
:width:`700px`
:label:`fig_p2x`

请注意，您应该使用带有合适驱动程序和支持 GPU 的深度学习框架的 GPU 实例。否则，使用 GPU 不会带来任何好处。

我们继续选择用于访问
实例的密钥对。如果您没有密钥对，请在 :numref:`fig_keypair` 中点击“Create new key pair”生成一个密钥对。随后，
您可以选择
之前生成的密钥对。
如果您生成了一个新的密钥对，请确保下载并将其存储在一个安全的位置。这是您 SSH 登录服务器的唯一方式。

![选择密钥对。](../img/keypair.png)
:width:`500px`
:label:`fig_keypair`

在此示例中，我们将保持“网络设置”的默认配置（点击“编辑”按钮以配置子网和安全组等项目）。我们只将默认硬盘大小增加到 64 GB (:numref:`fig_disk`)。请注意，CUDA 本身已经占用了 4 GB。

![修改硬盘大小。](../img/disk.png)
:width:`700px`
:label:`fig_disk`


点击“Launch Instance”以启动创建的
实例。点击
:numref:`fig_launching` 中显示的实例 ID 以查看该实例的状态。

![点击实例 ID。](../img/launching.png)
:width:`700px`
:label:`fig_launching`

### 连接到实例

如 :numref:`fig_connect` 所示，当实例状态变为绿色时，右键单击实例并选择 `Connect` 以查看实例访问方法。

![查看实例访问方法。](../img/connect.png)
:width:`700px`
:label:`fig_connect`

如果是新密钥，它必须不公开可见才能使 SSH 工作。转到存储 `D2L_key.pem` 的文件夹，并
执行以下命令
使密钥不公开可见：

```bash
chmod 400 D2L_key.pem
```

![查看实例访问和启动方法。](../img/chmod.png)
:width:`400px`
:label:`fig_chmod`


现在，复制 :numref:`fig_chmod` 下方红色框中的 SSH 命令并粘贴到命令行中：

```bash
ssh -i "D2L_key.pem" ubuntu@ec2-xx-xxx-xxx-xxx.y.compute.amazonaws.com
```

当命令行提示“Are you sure you want to continue connecting (yes/no)”时，输入“yes”并按 Enter 键登录实例。

您的服务器现在已经准备好了。


## 安装 CUDA

在安装 CUDA 之前，请确保更新实例以获得最新的驱动程序。

```bash
sudo apt-get update && sudo apt-get install -y build-essential git libgfortran3
```

这里我们下载 CUDA 12.1。访问 NVIDIA 的 [官方仓库](https://developer.nvidia.com/cuda-toolkit-archive) 以找到下载链接，如 :numref:`fig_cuda` 所示。

![查找 CUDA 12.1 的下载地址。](../img/cuda121.png)
:width:`500px`
:label:`fig_cuda`

复制指令并粘贴到终端以安装 CUDA 12.1。

```bash
# 链接和文件名可能会更改
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda-repo-ubuntu2204-12-1-local_12.1.0-530.30.02-1_amd64.deb
sudo dpkg -i cuda-repo-ubuntu2204-12-1-local_12.1.0-530.30.02-1_amd64.deb
sudo cp /var/cuda-repo-ubuntu2204-12-1-local/cuda-*-keyring.gpg /usr/share/keyrings/
sudo apt-get update
sudo apt-get -y install cuda
```

安装程序后，运行以下命令以查看 GPU：

```bash
nvidia-smi
```

最后，将 CUDA 添加到库路径中，以帮助其他库找到它，例如在 `~/.bashrc` 的末尾添加以下行。

```bash
export PATH="/usr/local/cuda-12.1/bin:$PATH"
export LD_LIBRARY_PATH=${LD_LIBRARY_PATH}:/usr/local/cuda-12.1/lib64
```

## 安装运行代码所需的库

要运行本书的代码，
只需按照 :ref:`chap_installation`
中的步骤为 Linux 用户在 EC2 实例上操作
并在远程 Linux 服务器上工作时使用以下提示：

* 要下载 Miniconda 安装页面上的 bash 脚本，右键单击下载链接并选择“复制链接地址”，然后执行 `wget [复制的链接地址]`。
* 在运行 `~/miniconda3/bin/conda init` 之后，您可以执行 `source ~/.bashrc` 而不是关闭并重新打开当前 shell。


## 远程运行 Jupyter Notebook

要远程运行 Jupyter Notebook，您需要使用 SSH 端口转发。毕竟，云中的服务器没有显示器或键盘。为此，请从您的桌面（或笔记本电脑）登录到服务器，如下所示：

```
# 此命令必须在本地命令行中运行
ssh -i "/path/to/key.pem" ubuntu@ec2-xx-xxx-xxx-xxx.y.compute.amazonaws.com -L 8889:localhost:8888
```

接下来，转到 EC2 实例上下载的本书代码的位置，
然后运行：

```
conda activate d2l
jupyter notebook
```

:numref:`fig_jupyter` 显示了运行 Jupyter Notebook 后可能的输出。最后一行是端口 8888 的 URL。

![运行 Jupyter Notebook 后的输出。最后一行是端口 8888 的 URL。](../img/jupyter.png)
:width:`700px`
:label:`fig_jupyter`

由于您使用了端口转发到 8889，
复制 :numref:`fig_jupyter` 中红色框的最后一行，
将 URL 中的“8888”替换为“8889”，
并在本地浏览器中打开它。


## 关闭未使用的实例

由于云服务是按使用时间计费的，因此您应关闭未使用的实例。请注意，还有其他选择：

* “停止”实例意味着您以后可以再次启动它。这类似于关闭常规服务器的电源。但是，停止的实例仍会因保留的硬盘空间而被收取少量费用。
* “终止”实例将删除与其关联的所有数据。这包括磁盘，因此您不能再次启动它。只有在确定将来不再需要它时才这样做。

如果您想将该实例用作许多其他实例的模板，
在 :numref:`fig_connect` 中右键单击示例并选择“Image”$\rightarrow$
“Create”以创建实例的镜像。完成后，选择
“Instance State”$\rightarrow$“Terminate”以终止实例。下次
您想使用此实例时，可以按照本节中的步骤
基于保存的镜像创建实例。唯一的区别是，在 :numref:`fig_ubuntu` 中的“1. Choose AMI”中，您必须使用左侧的“My AMIs”选项来选择您的保存镜像。创建的实例将保留镜像硬盘上存储的信息。例如，您不必重新安装 CUDA 和其他运行时环境。


## 总结

* 我们可以根据需要启动和停止实例，而不必购买和构建自己的计算机。
* 在使用支持 GPU 的深度学习框架之前，我们需要安装 CUDA。
* 我们可以使用端口转发在远程服务器上运行 Jupyter Notebook。


## 练习

1. 云计算提供了便利，但并不便宜。了解如何启动[竞价型实例](https://aws.amazon.com/ec2/spot/)以查看如何降低成本。
1. 尝试使用不同的 GPU 服务器。它们有多快？
1. 尝试使用多 GPU 服务器。您能多大程度地扩展性能？


[讨论](https://discuss.d2l.ai/t/423)